# Kiểm tra An toàn của Hợp đồng Marlowe

Mặc dù ngôn ngữ Marlowe có đảm bảo an toàn mạnh mẽ, nhưng vẫn có khả năng tạo ra các hợp đồng Marlowe không an toàn trên blockchain Cardano vì một số hợp đồng Marlowe hợp lệ vi phạm quy tắc sổ cái của blockchain.

Có ba công cụ Marlowe có sẵn để kiểm tra tính an toàn của một hợp đồng trước khi gửi nó lên blockchain:
- Điểm cuối REST của Marlowe Runtime `POST /contracts` trả về một báo cáo an toàn trong phản hồi của nó.
- Lệnh CLI của Marlowe Runtime `marlowe-runtime-cli create` in ra báo cáo an toàn.
- Lệnh CLI của Marlowe `marlowe-cli run analyze` cũng in ra một báo cáo an toàn tương tự.

Trong hướng dẫn này, chúng tôi sử dụng phương pháp đầu tiên, nhưng tất cả ba phương pháp đều cho kết quả giống nhau.

***Lưu ý rằng Marlowe Runtime không ngăn chặn người dùng từ việc gửi các hợp đồng không an toàn. Việc xem xét các lỗi an toàn và quyết định liệu có gửi hợp đồng lên blockchain hay không là trách nhiệm của khách hàng.*** Các lỗi an toàn được đánh dấu là "fatal" để chỉ ra liệu các vấn đề có nghiêm trọng đến mức hợp đồng có hoạt động trên blockchain hay không.

*Ngoài ra, lưu ý rằng các công cụ phân tích an toàn của Marlowe chưa được kiểm định, vì vậy nên thử nghiệm tất cả các đường dẫn thực thi của một hợp đồng Marlowe trên mạng lưới thử nghiệm trước khi chạy hợp đồng trên mạng lưới chính.*

## Chuẩn bị 

Xem [Chuẩn bị](../../docs/preliminaries.md) để biết thông tin về cách thiết lập môi trường của mình để sử dụng hướng dẫn này.

Bước đầu tiên là kiểm tra xem chúng ta có tất cả các công cụ cần thiết và biến môi trường có sẵn cho sổ tay hay không.

In [1]:
export SCRIPTS=../../scripts
export KEYS=../../keys
source $SCRIPTS/check-tools-and-env.sh

########################
## Check CLI commands ##
########################

The following required programs are available in the shell:
  * jq
  * json2yaml
  * marlowe-cli
  * marlowe-runtime-cli
  * cardano-cli
  * cardano-address
  * cardano-wallet

#########################
## Check required envs ##
#########################

The following environment variables are available in the shell:
  * CARDANO_NODE_SOCKET_PATH = /extra
  * MARLOWE_RT_HOST = 192.168.0.12
  * MARLOWE_RT_PORT = 13700
  * MARLOWE_RT_WEBSERVER_HOST = 192.168.0.12
  * MARLOWE_RT_WEBSERVER_PORT = 13780
  * MARLOWE_RT_WEBSERVER_URL = http://192.168.0.12:13780
cat: ../../keys/faucet.address: No such file or directory
  * FAUCET_ADDR = 
  * FAUCET_SKEY = ../../keys/faucet.skey

###################
## Check Network ##
###################

CARDANO_SCAN_URL = https://cardanoscan.io
MARLOWE_SCAN_URL = https://mainnet.marlowescan.com


## Ví dụ đầu tiên

Chúng ta tạo một hợp đồng chứa nhiều lỗi trong các token, tên, và địa chỉ của nó.

In [2]:
yaml2json << EOI > contract-1.json
when:
- case:
    deposits: 1
    into_account:
      address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j  # A testnet address.
    of_token:
      currency_symbol: 14696a4676909f4e3cb1f2e60e2e08e5abed70caf5c02699be9711   # Policy ID too short.
      token_name: This token name is longer than the ledger rules allow.        # Token name too long.
    party:
      address: addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h       # A mainnet address.
  then: close
- case:
    choose_between:
    - from: 1
      to: 2
    for_choice:
      choice_name: A Choice
      choice_owner:
        role_token: This role name is longer than the ledger rules allow.       # Role name too long.
  then: close
timeout: $((1000 * (`date -u +%s` + 1 * 60 * 60)))
timeout_continuation: close
EOI

Bây giờ hãy tạo yêu cầu JSON cho điểm cuối `POST /contracts` của Marlowe Runtime.

In [3]:
yaml2json << EOI > request-1.json
version: v1
contract: $(cat contract-1.json)
minUTxODeposit: 1200000
roles:
    An Extra Role: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j  # Not present in the contract.
metadata: {}
tags: {}
EOI
cat request-1.json | jq

{
  "contract": {
    "timeout": 1693257312000,
    "timeout_continuation": "close",
    "when": [
      {
        "case": {
          "deposits": 1,
          "into_account": {
            "address": "addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j"
          },
          "of_token": {
            "currency_symbol": "14696a4676909f4e3cb1f2e60e2e08e5abed70caf5c02699be9711",
            "token_name": "This token name is longer than the ledger rules allow."
          },
          "party": {
            "address": "addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h"
          }
        },
        "then": "close"
      },
      {
        "case": {
          "choose_between": [
            {
              "from": 1,
              "to": 2
            }
          ],
          "for_choice": {
            "choice_name": "A Choice",
            "choice_owner": {
              "role_token": "This role name is longer than the ledger rules allow."
            }
          }
 

Bây giờ gọi điểm cuối `POST /contracts` và nhận phản hồi, trong đó chứa một số báo cáo về lỗi an toàn.

In [4]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j" \
  -d @request-1.json \
  -o response-1.json \
  -sS
json2yaml response-1.json

links:
  contract: contracts/860dbbf717702fbb9ff31194235ef20fd51be839fff0d68ca8763045b38d211e%231
resource:
  contractId: 860dbbf717702fbb9ff31194235ef20fd51be839fff0d68ca8763045b38d211e#1
  safetyErrors:
  - detail: This role name is present in the contract, but that role token was not
      specified for minting.
    error: MissingRoleToken
    fatal: true
    role-name: This role name is longer than the ledger rules allow.
  - detail: This role token was specified for minting, but that role is not present
      in the contract.
    error: ExtraRoleToken
    fatal: false
    role-name: An Extra Role
  - detail: This role name is longer than the 32 bytes allowed by the ledger rules.
    error: RoleNameTooLong
    fatal: true
    role-name: This role name is longer than the ledger rules allow.
  - currency-symbol: 14696a4676909f4e3cb1f2e60e2e08e5abed70caf5c02699be9711
    detail: This currency symbol is not the 28-bytes required by the ledger rules.
    error: InvalidCurrencySymbol
   

Có bảy lỗi an toàn, chúng ta sẽ xem xét từng lỗi một.

In [5]:
jq -r '.resource.safetyErrors | .[] | .error' response-1.json

MissingRoleToken
ExtraRoleToken
RoleNameTooLong
InvalidCurrencySymbol
WrongNetwork
InconsistentNetworks
TransactionValidationError


### `MissingRoleToken`

Tên vai trò `This role name is longer than the ledger rules allow.` xuất hiện trong hợp đồng, nhưng yêu cầu tạo không đề cập đến vai trò đó.

In [6]:
jq '.resource.safetyErrors[0]' response-1.json | json2yaml

detail: This role name is present in the contract, but that role token was not specified
  for minting.
error: MissingRoleToken
fatal: true
role-name: This role name is longer than the ledger rules allow.


### `ExtraRoleToken`

Tên vai trò `An Extra Role` được đề cập trong yêu cầu tạo, nhưng không được sử dụng trong hợp đồng chính.

In [7]:
jq '.resource.safetyErrors[1]' response-1.json | json2yaml

detail: This role token was specified for minting, but that role is not present in
  the contract.
error: ExtraRoleToken
fatal: false
role-name: An Extra Role


### `RoleNameTooLong`

Vai trò `This role name is longer than the ledger rules allow.` vượt quá giới hạn 32 byte được cho phép bởi các quy tắc sổ cái.

In [8]:
jq '.resource.safetyErrors[2]' response-1.json | json2yaml

detail: This role name is longer than the 32 bytes allowed by the ledger rules.
error: RoleNameTooLong
fatal: true
role-name: This role name is longer than the ledger rules allow.


### `InvalidCurrencySymbol`

Các byte hexadecimals `14696a4676909f4e3cb1f2e60e2e08e5abed70caf5c02699be9711` không phải là một biểu tượng tiền tệ hợp lệ, vì chúng quá ngắn.

In [9]:
jq '.resource.safetyErrors[3]' response-1.json | json2yaml

currency-symbol: 14696a4676909f4e3cb1f2e60e2e08e5abed70caf5c02699be9711
detail: This currency symbol is not the 28-bytes required by the ledger rules.
error: InvalidCurrencySymbol
fatal: true


### `WrongNetwork`

Hợp đồng đề cập đến một địa chỉ `mainnet`, nhưng Marlowe Runtime đang kết nối với mạng thử nghiệm `preprod`.

In [10]:
jq '.resource.safetyErrors[4]' response-1.json | json2yaml

detail: The contract contains addresses that are do not match the network on which
  it will be executed.
error: WrongNetwork
fatal: true


### `InconsistentNetworks`

Hợp đồng chứa một sự kết hợp của các địa chỉ trên mainnet và testnet.

In [11]:
jq '.resource.safetyErrors[5]' response-1.json | json2yaml

detail: The contract contains both mainnet and testnet addresses.
error: InconsistentNetworks
fatal: true


### `TransactionValidationError`

Giao dịch tạo sẽ không được xác nhận trên nút nếu được gửi. Nút sẽ không cung cấp một lý do chi tiết về tại sao giao dịch không được xác nhận, vì vậy Marlowe Runtime cũng không thể cung cấp. Trong trường hợp cụ thể này, sự tồn tại của token không hợp lệ là nguyên nhân gây ra sự cố. Báo cáo an toàn liệt kê tất cả các chi tiết của giao dịch tương lai tiềm năng mà sẽ gặp sự cố xác nhận.

In [12]:
jq '.resource.safetyErrors[6]' response-1.json | json2yaml

detail: This transaction fails to validate on the ledger.
error: TransactionValidationError
fatal: true
message: ToCardanoError
transaction:
  contract:
    timeout: 1693257312000
    timeout_continuation: close
    when:
    - case:
        deposits: 1
        into_account:
          address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j
        of_token:
          currency_symbol: 14696a4676909f4e3cb1f2e60e2e08e5abed70caf5c02699be9711
          token_name: This token name is longer than the ledger rules allow.
        party:
          address: addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h
      then: close
    - case:
        choose_between:
        - from: 1
          to: 2
        for_choice:
          choice_name: A Choice
          choice_owner:
            role_token: This role name is longer than the ledger rules allow.
      then: close
  input:
    tx_inputs:
    - input_from_party:
        address: addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pv

## Ví dụ 2

Chúng ta tạo một hợp đồng có một số vấn đề an toàn khác.

In [13]:
yaml2json << EOI > contract-2.json
when:
- case:
    deposits: 1
    into_account:
      address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j
    of_token:
      currency_symbol: c02699be97113914696a4676909f4e3cb1f2e60e2e08e5abed70caf5
      token_name: This token name is longer than the ledger rules allow.  # Token name too long.
    party:
      role_token: ''
  then: close
- case:
    deposits: 1
    into_account:
      address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j
    of_token:
      currency_symbol: ''
      token_name: Impossible to mint.
    party:
      role_token: ''
  then: close
timeout: $((1000 * (`date -u +%s` + 1 * 60 * 60)))
timeout_continuation: close
EOI

Bây giờ hãy tạo yêu cầu JSON cho điểm cuối `POST /contracts` của Marlowe Runtime.

In [14]:
yaml2json << EOI > request-2.json
version: v1
contract: $(cat contract-2.json)
minUTxODeposit: 1200000
#roles: 14696a4676909f4e3cb1f2e60e2e08e5abed70caf5c02699be971139
metadata: {}
tags: {}
EOI
cat request-2.json | jq

{
  "contract": {
    "timeout": 1693257317000,
    "timeout_continuation": "close",
    "when": [
      {
        "case": {
          "deposits": 1,
          "into_account": {
            "address": "addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j"
          },
          "of_token": {
            "currency_symbol": "c02699be97113914696a4676909f4e3cb1f2e60e2e08e5abed70caf5",
            "token_name": "This token name is longer than the ledger rules allow."
          },
          "party": {
            "role_token": ""
          }
        },
        "then": "close"
      },
      {
        "case": {
          "deposits": 1,
          "into_account": {
            "address": "addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j"
          },
          "of_token": {
            "currency_symbol": "",
            "token_name": "Impossible to mint."
          },
          "party": {
            "role_token": ""
          }
        },
        "then": "close"
     

Yêu cầu Marlowe Runtime xây dựng giao dịch tạo dẫn đến năm lỗi an toàn. Chúng ta sẽ thảo luận về ba lỗi đầu tiên.

In [15]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j" \
  -d @request-2.json \
  -o response-2.json \
  -sS
jq -r '.resource.safetyErrors | .[] | .error' response-2.json

MissingRolesCurrency
InvalidToken
TokenNameTooLong
TransactionValidationError
TransactionValidationError


### `MissingRolesCurrency`

Chúng ta đã không chỉ định tiền tệ của các vai trò trong phần thân yêu cầu.

In [16]:
jq '.resource.safetyErrors[0]' response-2.json | json2yaml

detail: Roles are present in the contract, but no roles currency was specified.
error: MissingRolesCurrency
fatal: true


### `InvalidToken`

Không thể tồn tại một token không phải ADA cho biểu tượng tiền tệ trống.

In [17]:
jq '.resource.safetyErrors[1]' response-2.json | json2yaml

detail: This token associates a name with the ada currency symbol.
error: InvalidToken
fatal: true
token:
  currency_symbol: ''
  token_name: Impossible to mint.


### `TokenNameTooLong`

Tên `This token name is longer than the 32 bytes allowed by the ledger rules.` quá dài cho một token, vượt quá giới hạn 32 byte được cho phép bởi các quy tắc sổ cái.

In [18]:
jq '.resource.safetyErrors[2]' response-2.json | json2yaml

detail: This token name is longer than the 32 bytes allowed by the ledger rules.
error: TokenNameTooLong
fatal: true
token-name: This token name is longer than the ledger rules allow.


## Ví dụ 3

Chúng ta sử dụng hợp đồng từ ví dụ trước, nhưng giao dịch tạo chỉ định một ID chính sách tiền tệ của vai trò quá ngắn.

In [19]:
yaml2json << EOI > request-3.json
version: v1
contract: $(cat contract-2.json)
minUTxODeposit: 1200000
roles: a4676909f4e3cb1f2e60e2e08e5abed70caf5c02699be971139  # Not a valid policy ID.
metadata: {}
tags: {}
EOI
cat request-3.json | jq

{
  "contract": {
    "timeout": 1693257317000,
    "timeout_continuation": "close",
    "when": [
      {
        "case": {
          "deposits": 1,
          "into_account": {
            "address": "addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j"
          },
          "of_token": {
            "currency_symbol": "c02699be97113914696a4676909f4e3cb1f2e60e2e08e5abed70caf5",
            "token_name": "This token name is longer than the ledger rules allow."
          },
          "party": {
            "role_token": ""
          }
        },
        "then": "close"
      },
      {
        "case": {
          "deposits": 1,
          "into_account": {
            "address": "addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j"
          },
          "of_token": {
            "currency_symbol": "",
            "token_name": "Impossible to mint."
          },
          "party": {
            "role_token": ""
          }
        },
        "then": "close"
     

Một loại tiền tệ của vai trò không hợp lệ dẫn đến một phản hồi lỗi HTTP.

In [20]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j" \
  -d @request-3.json \
  -iSs

HTTP/1.1 400 Bad Request
Transfer-Encoding: chunked
Date: Mon, 28 Aug 2023 20:15:20 GMT
Server: Warp/3.3.24

Error in $.roles: invalid bytestring size

## Ví dụ 4

Chúng ta tạo hợp đồng đơn giản nhất, không có vai trò hoặc hoạt động khác.

In [21]:
cat << EOI > contract-4.json
"close"
EOI

Bây giờ hãy tạo yêu cầu JSON cho điểm cuối `POST /contracts` của Marlowe Runtime.

In [22]:
yaml2json << EOI > request-4.json
version: v1
contract: $(cat contract-4.json)
minUTxODeposit: 1200000
roles: 14696a4676909f4e3cb1f2e60e2e08e5abed70caf5c02699be971139
metadata: {}
tags: {}
EOI
cat request-4.json | jq

{
  "contract": "close",
  "metadata": {},
  "minUTxODeposit": 1200000,
  "roles": "14696a4676909f4e3cb1f2e60e2e08e5abed70caf5c02699be971139",
  "tags": {},
  "version": "v1"
}


Yêu cầu Marlowe Runtime xây dựng giao dịch tạo dẫn đến một lỗi an toàn.

In [23]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j" \
  -d @request-4.json \
  -o response-4.json \
  -Ss
jq -r '.resource.safetyErrors | .[] | .error' response-4.json

ContractHasNoRoles


### `ContractHasNoRoles`

Một biểu tượng tiền tệ của các vai trò đã được chỉ định cho một hợp đồng không chứa bất kỳ vai trò nào.

In [24]:
jq '.resource.safetyErrors[0]' response-4.json | json2yaml

detail: No roles are present in the contract, but a roles currency was specified.
error: ContractHasNoRoles
fatal: false


## Ví dụ 5

Chúng ta tạo một hợp đồng tuân theo các quy tắc sổ cái nhưng tạo ra cảnh báo do quy tắc ngữ nghĩa Marlowe: trong trường hợp này, hợp đồng cố gắng thanh toán nhiều ada hơn đã được gửi. Hợp đồng này sẽ chạy trên blockchain, nhưng các cảnh báo cho thấy rằng nó có thể chứa các lỗi thiết kế hoặc không tuân thủ các quy tắc thiết kế tốt nhất cho các hợp đồng Marlowe.

In [25]:
yaml2json << EOI > contract-5.json
when:
- case:
    deposits: 10000000
    into_account:
      role_token: My Role
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      role_token: My Role
  then:
    from_account:
      role_token: My Role
    pay: 20000000
    then: close
    to:
      party:
        role_token: My Role
    token:
      currency_symbol: ''
      token_name: ''
timeout: $((1000 * (`date -u +%s` + 1 * 60 * 60)))
timeout_continuation: close
EOI

Bây giờ hãy tạo yêu cầu JSON cho điểm cuối `POST /contracts` của Marlowe Runtime.

In [26]:
yaml2json << EOI > request-5.json
version: v1
contract: $(cat contract-5.json)
minUTxODeposit: 1200000
roles: 99be97113914696a4676909f4e3cb1f2e60e2e08e5abed70caf5c026
metadata: {}
tags: {}
EOI
cat request-5.json | jq

{
  "contract": {
    "timeout": 1693257323000,
    "timeout_continuation": "close",
    "when": [
      {
        "case": {
          "deposits": 10000000,
          "into_account": {
            "role_token": "My Role"
          },
          "of_token": {
            "currency_symbol": "",
            "token_name": ""
          },
          "party": {
            "role_token": "My Role"
          }
        },
        "then": {
          "from_account": {
            "role_token": "My Role"
          },
          "pay": 20000000,
          "then": "close",
          "to": {
            "party": {
              "role_token": "My Role"
            }
          },
          "token": {
            "currency_symbol": "",
            "token_name": ""
          }
        }
      }
    ]
  },
  "metadata": {},
  "minUTxODeposit": 1200000,
  "roles": "99be97113914696a4676909f4e3cb1f2e60e2e08e5abed70caf5c026",
  "tags": {},
  "version": "v1"
}


Yêu cầu Marlowe Runtime xây dựng giao dịch tạo dẫn đến một lỗi an toàn.

In [27]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j" \
  -d @request-5.json \
  -o response-5.json \
  -sS
jq -r '.resource.safetyErrors | .[] | .error' response-5.json

TransactionWarning


### `TransactionWarning`

Một số áp dụng của các đầu vào vào hợp đồng có thể kích hoạt một cảnh báo từ ngữ nghĩa Marlowe. Một cảnh báo như vậy cũng sẽ được hiển thị trên tab "Cảnh báo" của Marlowe Playground. Báo cáo an toàn liệt kê tất cả các chi tiết của giao dịch tiềm năng trong tương lai mà sẽ xuất hiện cảnh báo.

In [28]:
jq '.resource.safetyErrors[0]' response-5.json | json2yaml

detail: A Marlowe semantics warning is reported for this transaction.
error: TransactionWarning
fatal: false
transaction:
  contract:
    timeout: 1693257323000
    timeout_continuation: close
    when:
    - case:
        deposits: 10000000
        into_account:
          role_token: My Role
        of_token:
          currency_symbol: ''
          token_name: ''
        party:
          role_token: My Role
      then:
        from_account:
          role_token: My Role
        pay: 20000000
        then: close
        to:
          party:
            role_token: My Role
        token:
          currency_symbol: ''
          token_name: ''
  input:
    tx_inputs:
    - input_from_party:
        role_token: My Role
      into_account:
        role_token: My Role
      of_token:
        currency_symbol: ''
        token_name: ''
      that_deposits: 10000000
    tx_interval:
      from: 0
      to: 0
  output:
    contract: close
    payments:
    - amount: 10000000
      payment_from:


## Ví dụ 6

Chúng ta tạo một hợp đồng có thể vượt quá giới hạn chi phí thực thi Plutus trên blockchain Cardano.

In [29]:
yaml2json << EOI > contract-6.json
timeout: $((1000 * ($(date -u +%s) + 1 * 60 * 60)))
timeout_continuation: close
when:
- case:
    deposits: 5000000
    into_account:
      role_token: Alice
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j
  then:
    timeout: $((1000 * (`date -u +%s + 2 * 60 * 60)))
    timeout_continuation: close
    when:
    - case:
        deposits: 5000000
        into_account:
          role_token: Bob
        of_token:
          currency_symbol: ''
          token_name: ''
        party:
          address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j
      then:
        timeout: $((1000 * (`date -u +%s + 3 * 60 * 60)))
        timeout_continuation: close
        when:
        - case:
            deposits: 5000000
            into_account:
              role_token: Charlie
            of_token:
              currency_symbol: ''
              token_name: ''
            party:
              address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j
          then:
            timeout: $((1000 * (`date -u +%s + 4 * 60 * 60)))
            timeout_continuation: close
            when:
            - case:
                deposits: 5000000
                into_account:
                  role_token: Dave
                of_token:
                  currency_symbol: ''
                  token_name: ''
                party:
                  address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j
              then:
                timeout: $((1000 * (`date -u +%s + 5 * 60 * 60)))
                timeout_continuation: close
                when:
                - case:
                    deposits: 5000000
                    into_account:
                      role_token: Eve
                    of_token:
                      currency_symbol: ''
                      token_name: ''
                    party:
                      address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j
                  then:
                    timeout: $((1000 * (`date -u +%s + 6 * 60 * 60)))
                    timeout_continuation: close
                    when:
                    - case:
                        notify_if: true
                      then: close
EOI

Bây giờ hãy tạo yêu cầu JSON cho điểm cuối `POST /contracts` của Marlowe Runtime.

In [30]:
yaml2json << EOI > request-6.json
version: v1
contract: $(cat contract-6.json)
minUTxODeposit: 1200000
roles: 99be97113914696a4676909f4e3cb1f2e60e2e08e5abed70caf5c026
metadata: {}
tags: {}
EOI
cat request-6.json | jq

{
  "contract": {
    "timeout": 1693257328000,
    "timeout_continuation": "close",
    "when": [
      {
        "case": {
          "deposits": 5000000,
          "into_account": {
            "role_token": "Alice"
          },
          "of_token": {
            "currency_symbol": "",
            "token_name": ""
          },
          "party": {
            "address": "addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j"
          }
        },
        "then": {
          "timeout": 1693260928000,
          "timeout_continuation": "close",
          "when": [
            {
              "case": {
                "deposits": 5000000,
                "into_account": {
                  "role_token": "Bob"
                },
                "of_token": {
                  "currency_symbol": "",
                  "token_name": ""
                },
                "party": {
                  "address": "addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j"
     

Yêu cầu Marlowe Runtime xây dựng giao dịch tạo dẫn đến các lỗi an toàn.

In [31]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j" \
  -d @request-6.json \
  -o response-6.json \
  -sS
jq -r '.resource.safetyErrors | .[] | .error' response-6.json

TransactionValidationError
TransactionValidationError


### `TransactionValidationError`

Chúng ta đã thấy lỗi này trước đó trong ví dụ đầu tiên khi nút Cardano đã từ chối giao dịch trong quá trình xác nhận giai đoạn một. Ở đây, chúng ta có một lỗi xác nhận giai đoạn hai (Plutus) vì một giao dịch quá tốn kém để thực thi. Lưu ý rằng giao dịch thất bại không phải là giao dịch tạo, mà là một giao dịch trong các giai đoạn sau của việc thực thi hợp đồng. Báo cáo an toàn liệt kê tất cả các chi tiết của giao dịch tiềm năng trong tương lai mà sẽ xuất hiện lỗi xác nhận.

In [32]:
jq '.resource.safetyErrors | .[0]' response-6.json

{
  "detail": "This transaction fails to validate on the ledger.",
  "error": "TransactionValidationError",
  "fatal": true,
  "message": "BalancingError \"TxBodyScriptExecutionError [(ScriptWitnessIndexTxIn 1,ScriptErrorEvaluationFailed (CekError An error has occurred:  User error:\\nThe machine terminated part way through evaluation due to overspending the budget.\\nThe budget when the machine terminated was:\\n({ cpu: 6343692948\\n| mem: -1158\\n})\\nNegative numbers indicate the overspent budget; note that this only indicatessthe budget that was needed for the next step, not to run the program to completion.) [\\\"Data decoded successfully\\\",\\\"Redeemer decoded successfully\\\",\\\"Script context decoded successfully\\\"])]\"",
  "transaction": {
    "contract": {
      "timeout": 1693275328000,
      "timeout_continuation": "close",
      "when": [
        {
          "case": {
            "notify_if": true
          },
          "then": "close"
        }
      ]
    },
    "in

Trường `.message` của lỗi an toàn cung cấp thông tin từ trình diễn Plutus.

In [33]:
jq '.resource.safetyErrors | .[0].message' response-6.json

"BalancingError \"TxBodyScriptExecutionError [(ScriptWitnessIndexTxIn 1,ScriptErrorEvaluationFailed (CekError An error has occurred:  User error:\\nThe machine terminated part way through evaluation due to overspending the budget.\\nThe budget when the machine terminated was:\\n({ cpu: 6343692948\\n| mem: -1158\\n})\\nNegative numbers indicate the overspent budget; note that this only indicatessthe budget that was needed for the next step, not to run the program to completion.) [\\\"Data decoded successfully\\\",\\\"Redeemer decoded successfully\\\",\\\"Script context decoded successfully\\\"])]\""


### `SafetyAnalysisTimeout`

Lưu ý rằng một lỗi `SafetyAnalysisTimeout` có thể được báo cáo nếu hợp đồng Marlowe quá lớn đến mức máy chủ Marlowe Runtime hết thời gian phân tích hợp đồng. (Marlowe Runtime áp đặt một giới hạn thời gian cho phân tích an toàn để ngăn chặn các cuộc tấn công từ chối dịch vụ.) Trong tình huống này, bạn có thể sử dụng `marlowe-cli run analyze` để phân tích hợp đồng vì `marlowe-cli` không áp đặt giới hạn thời gian cho phân tích.

## Các lỗi an toàn khác

Các ví dụ trên đã minh họa tất cả các lỗi an toàn Marlowe phổ biến mà Marlowe Runtime có thể phát hiện.

Nếu hợp đồng được xây dựng bằng cách sử dụng `marlowe-cli` thay vì Marlowe Runtime, thì có thể xảy ra một số lỗi an toàn bổ sung. Một số trong số này liên quan đến trạng thái Marlowe ban đầu, mà Marlowe Runtime xây dựng một cách an toàn nhưng không bị ràng buộc bởi Marlowe CLI.
- `NonPositiveBalance`: Số dư tài khoản ban đầu không dương.
- `DuplicateAccount`: Một mục tài khoản trùng lặp tồn tại trong trạng thái Marlowe ban đầu.
- `DuplicateChoice`: Một mục lựa chọn trùng lặp tồn tại trong trạng thái Marlowe ban đầu.
- `DuplicateBoundValue`: Một mục giá trị ràng buộc trùng lặp tồn tại trong trạng thái Marlowe ban đầu.

Các lỗi an toàn khác liên quan đến phân tích tiên lượng được thực hiện bởi Marlowe CLI nhưng không bởi Marlowe Runtime. (Chúng không cần thiết cho Marlowe Runtime vì Marlowe Runtime luôn cố gắng thực thi mọi giao dịch có thể cho hợp đồng và vì vậy sẽ phát hiện ra những lỗi này. Trong Marlowe CLI, việc thực thi mọi giao dịch là tùy chọn.)
- `MaximumValueMayExceedProtocol`: Có thể có quá nhiều token được lưu trữ tại một số điểm trong hợp đồng.
- `TransactionSizeMayExceedProtocol`: Kích thước giao dịch (theo byte) có thể quá lớn.
- `TransactionCostMayExceedProtocol` : Chi phí thực thi của giao dịch có thể quá cao.
- `MissingContinuation`: Hợp đồng thiếu một phần tiếp theo không có trong bản đồ tiếp theo của nó.